In [42]:
import math

import torch
import torch.nn as nn

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
from sklearn.model_selection import train_test_split

In [43]:
x = pd.read_csv('./data/train_data.csv').drop(['date'], axis=1).values[:, 1:]
y = pd.read_csv('./data/train_target.csv').values[:,1]

x_validate = pd.read_csv('./data/test_data.csv').drop(['date'], axis=1).values[:, 1:]

In [44]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [45]:
# dataset_train = TensorDataset(torch.tensor(x_train, dtype=torch.float64), torch.tensor(y_train, dtype=torch.float64))
# dataset_test = TensorDataset(torch.tensor(x_test, dtype=torch.float64), torch.tensor(y_test, dtype=torch.float64))

# dataloader_train = DataLoader(dataset_train, batch_size=128)
# dataloader_test = DataLoader(dataset_test, batch_size=128)

In [46]:
# net = nn.Sequential(
#     nn.Linear(18, 10), 
#     nn.ReLU(),
#     nn.Linear(10, 5),
#     nn.ReLU(),
#     nn.Linear(5, 1)
# ).to(dtype=torch.float64)

# net = nn.Sequential(
#     nn.Linear(18, 1)
# ).to(dtype=torch.float64)

In [47]:
# optim = Adam(net.parameters(), lr=0.001)
# loss_func = nn.MSELoss()

In [48]:
# for _ in range(10):
#     for x_batch, y_batch in dataloader_train:
#         y_batch_predict = net(x_batch)
#         loss = (y_batch_predict - y_batch).pow(2).mean()
#         loss.backward()
#         optim.step()
#         optim.zero_grad()
        
#     loss_train = 0.
#     loss_test = 0.
#     n_train = len(dataloader_train) * dataloader_train.batch_size
#     n_test = len(dataloader_test) * dataloader_test.batch_size
#     for x_batch, y_batch in dataloader_train:
#         y_batch_predict = net(x_batch)
#         loss_train += (y_batch_predict - y_batch).pow(2).sum().item()
        
#     for x_batch, y_batch in dataloader_test:
#         y_batch_predict = net(x_batch)
#         loss_test += (y_batch_predict - y_batch).pow(2).sum().item()
#     print(math.sqrt(loss_train / n_train), math.sqrt(loss_test / n_test))

In [49]:
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, Normalizer, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor

In [50]:
# preprocess = lambda x: PolynomialFeatures(degree=2).fit_transform(MinMaxScaler().fit_transform(x))
preprocess = lambda x: MinMaxScaler().fit_transform(x)

In [51]:
x_train_pr, x_test_pr, x_validate_pr = preprocess(x_train), preprocess(x_test), preprocess(x_validate)

In [58]:
model = XGBRegressor(n_estimators=500, learning_rate=0.07, max_depth=4, n_jobs=16)
# model = LinearRegression(n_jobs=16)

In [59]:
model.fit(x_train_pr, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.07, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=16, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [60]:
(sum((model.predict(x_test_pr) - y_test) ** 2) / y_test.shape[0]) ** (1/2)

135.95400834479625

In [56]:
y_validate = model.predict(x_validate_pr)

In [57]:
with open('submission.csv', 'w') as file:
    file.write('index,price\n')
    for idx in range(y_validate.shape[0]):
        file.write(str(int(x_validate[idx, 0])) + ',' + str(y_validate[idx]) + '\n')